<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук ВШЭ <br> Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 3. Обучение с учителем. Методы классификации
## <center>Часть 5. Дерево решений и случайный лес <br> в соревновании Kaggle Inclass по кредитному скорингу

**[Соревнование](https://inclass.kaggle.com/c/beeline-credit-scoring-competition-3).**

Решается задача кредитного скоринга. 

Признаки клиентов банка:
- Age - возраст (вещественный)
- Income - месячный доход (вещественный)
- BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
- DIR - Debt-to-income Ratio (вещественный)
- NumLoans - число заемов и кредитных линий
- NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
- NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
- Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
- Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
- Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Загружаем данные.**

In [2]:
train_df = pd.read_csv('../../data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('../../data/credit_scoring_test.csv', index_col='client_id')

In [3]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [4]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350


**Посчитаем число пропусков в каждом признаке.**

In [5]:
for col in train_df.columns:
    print("{0}, num. NA's: {1}".format(col, pd.isnull(train_df[col]).sum()))

DIR, num. NA's: 0
Age, num. NA's: 0
NumLoans, num. NA's: 0
NumRealEstateLoans, num. NA's: 0
NumDependents, num. NA's: 1916
Num30-59Delinquencies, num. NA's: 0
Num60-89Delinquencies, num. NA's: 0
Income, num. NA's: 14847
BalanceToCreditLimit, num. NA's: 0


In [6]:
for col in test_df.columns:
    print("{0}, num. NA's: {1}".format(col, pd.isnull(test_df[col]).sum()))

DIR, num. NA's: 0
Age, num. NA's: 0
NumLoans, num. NA's: 0
NumRealEstateLoans, num. NA's: 0
NumDependents, num. NA's: 2008
Num30-59Delinquencies, num. NA's: 0
Num60-89Delinquencies, num. NA's: 0
Income, num. NA's: 14884
BalanceToCreditLimit, num. NA's: 0


**Заменим пропуски медианными значениями.**

In [7]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

### Дерево решений без настройки параметров

In [8]:
first_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
first_tree.fit(train_df, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')

**Прогноз для тестовой выборки.**

In [9]:
first_tree_pred = first_tree.predict(test_df)

**Запишем прогноз в файл.**

In [10]:
def write_to_submission_file(predicted_labels, out_file,
                             target='Delinquent90', index_label="client_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(75000, 
                                                  predicted_labels.shape[0] + 75000),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [11]:
write_to_submission_file(first_tree_pred, '../../output/credit_scoring_first_tree.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки - 0.537 ROC AUC.**

**Если предсказывать вероятности дефолта для клиентов тестовой выборки, результат будет намного лучше.**

In [12]:
first_tree_pred_probs = first_tree.predict_proba(test_df)[:, 1]

In [13]:
write_to_submission_file(first_tree_pred_probs, '../../output/credit_scoring_first_tree_prob.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки - 0.804 ROC AUC.**

## Дерево решений с настройкой параметров с помощью GridSearch

In [14]:
tree_params = {'criterion': ('gini', 'entropy'), 
               'max_depth': list(range(1,11)), 
               'min_samples_leaf': list(range(1,11))}

locally_best_tree = GridSearchCV(DecisionTreeClassifier(random_state=42), 
                                 tree_params, 
                                 verbose=True, n_jobs=-1, cv=5,
                                scoring='roc_auc')
locally_best_tree.fit(train_df, y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  2.8min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'criterion': ('gini', 'entropy')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=True)

In [15]:
locally_best_tree.best_params_, round(locally_best_tree.best_score_, 3)

({'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 10},
 0.82999999999999996)

In [16]:
tuned_tree_pred_probs = locally_best_tree.predict_proba(test_df)[:, 1]

In [17]:
write_to_submission_file(tuned_tree_pred_probs, '../../output/credit_scoring_tuned_tree.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки - 0.836 ROC AUC.**

## Случайный лес без настройки параметров

In [18]:
first_forest = RandomForestClassifier(max_depth=3, random_state=42, n_jobs=-1)
first_forest.fit(train_df, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [19]:
first_forest_pred = first_forest.predict_proba(test_df)[:, 1]

In [20]:
write_to_submission_file(first_forest_pred, '../../output/credit_scoring_first_forest.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки - 0.834 ROC AUC.**

## Случайный лес c настройкой параметров

In [22]:
%%time
forest_params = {'max_depth': list(range(8,14)),
                 'min_samples_leaf': list(range(3,8))}

locally_best_forest = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), 
                                 forest_params, 
                                 verbose=True, n_jobs=-1, cv=5,
                                  scoring='roc_auc')
locally_best_forest.fit(train_df, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.3min finished


CPU times: user 5.25 s, sys: 882 ms, total: 6.13 s
Wall time: 1min 16s


In [23]:
locally_best_forest.best_params_, round(locally_best_forest.best_score_, 3)

({'max_depth': 9, 'min_samples_leaf': 5}, 0.84399999999999997)

In [24]:
tuned_forest_pred = locally_best_forest.predict_proba(test_df)[:, 1]

In [25]:
write_to_submission_file(tuned_forest_pred, '../../output/credit_scoring_tuned_forest.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки ~ 0.847 ROC AUC.**

**Обычно увеличение количества деревьев только улучшает результат. Так что напоследок обучим случайный лес из 300 деревьев с найденными лучшими параметрами.**

In [31]:
%%time
final_forest = RandomForestClassifier(n_estimators=300, max_depth=9, min_samples_leaf=5,
                                      random_state=42, n_jobs=-1, oob_score=True)
final_forest.fit(train_df, y)
print('OOB score: {}'.format(final_forest.oob_score_))
final_forest_pred = final_forest.predict_proba(test_df)[:, 1]
write_to_submission_file(final_forest_pred, '../../output/credit_scoring_final_forest.csv')

OOB score: 0.93516
CPU times: user 56 s, sys: 1.44 s, total: 57.5 s
Wall time: 18.7 s


**На публичной части тестовой выборки на Kaggle результат у этой посылки - 0.853 ROC AUC.**